<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/message_processing/SerializableMessage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Serializable Messages


When working with LangChain message types in our state schema, there are important considerations for serialization. We should use `AnyMessage` instead of `BaseMessage` for proper serialization/deserialization of message objects.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai

First, create a State object and a test node to test the message serialization:

In [2]:
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from typing import List

class ChatState(BaseModel):
    messages: List[AnyMessage]
    context: str

def add_message(state: ChatState):
    return {"messages": state.messages + [AIMessage(content="Hello there!")]}

Create graph state , add a node and compile the new graph model:

In [3]:
builder = StateGraph(ChatState)
builder.add_node("add_message", add_message)
builder.add_edge(START, "add_message")
builder.add_edge("add_message", END)
graph = builder.compile()

Next, create an input with a message and invoke the graph with this input:

In [4]:
initial_state = ChatState(
    messages = [HumanMessage(content="Hi")], context="Customer support chat"
)

result = graph.invoke(initial_state)
print(f"Output: {result}")

Output: {'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello there!', additional_kwargs={}, response_metadata={})], 'context': 'Customer support chat'}


Finally, convert back the result to Pydantic model to see the message types

In [5]:
output_model = ChatState(**result)
for i, msg in enumerate(output_model.messages):
    print(f'Message {i}: {type(msg).__name__} - {msg.content}')

Message 0: HumanMessage - Hi
Message 1: AIMessage - Hello there!
